# Working with the ScienceImage Class [v1]

In [32]:
# imports
import os
import numpy as np
from importlib import reload

In [3]:
# Path to PYPIT-Development-suite
pypdev_path = os.getenv('PYPIT_DEV')

----

# Development

In [80]:
from pypit.core import arprocimg
from pypit import biasframe
from pypit import flatfield
from pypit import processimages
from pypit import scienceimage
from pypit import traceslits
from pypit import waveimage
from pypit import wavetilts

## Settings

In [67]:
settings = dict(masters={})
settings['masters']['directory'] = pypdev_path+'/Cooked/MF_shane_kast_blue'
settings['masters']['reuse'] = True
settings['masters']['loaded'] = []
#
setup = 'A_01_aa'
#
settings['combine'] = {}

In [68]:
spectrograph = 'shane_kast_blue'
det = 1

In [123]:
settings['detector'] = {}
settings['detector']['num'] = det
settings['detector']['dataext'] = 0
settings['detector']['datasec01'] = [[0, 1024], [0, 0]]
settings['detector']['datasec02'] = [[1024, 2048], [0, 0]]
settings['detector']['oscansec01'] = [[2049, 2080], [0, 0]]
settings['detector']['oscansec02'] = [[2080, 2111], [0, 0]]
settings['detector']['naxis0'] = 2112  # Raw frame, with overscan
settings['detector']['naxis1'] = 350
settings['detector']['numamplifiers'] = 2
settings['detector']['gain'] = [1.2, 1.2]
settings['detector']['ronoise'] = [3.7, 3.7]
settings['detector']['saturation'] = 65535.
settings['detector']['nonlinear'] = 0.76
settings['detector']['dispaxis'] = 1
settings['detector']['darkcurr'] = 0.

In [237]:
settings['trace'] = {}
settings['trace']['object'] = {}
settings['trace']['object']['order'] = 2                # What is the order of the polynomial function to be used to fit the object trace in each slit
settings['trace']['object']['function'] = 'legendre'    # What function should be used to trace the object in each slit? (polynomial, legendre, chebyshev)
settings['trace']['object']['find'] = 'standard'         # What algorithm to use for finding objects [standard, nminima]
settings['trace']['object']['nsmooth'] = 3              # Parameter for Gaussian smoothing when the nminima algorithm is used
settings['trace']['object']['xedge'] = 0.03             # Ignore any objects within xedge of the edge of the slit

In [239]:
settings['science'] = {}
settings['science']['extraction'] = {}
settings['science']['extraction']['reuse'] = False          # If the science frame has previously been extracted and saved, load the extractions
settings['science']['extraction']['profile'] = 'gaussian'   # Fitting function used to extract science data, only if the extraction is 2D (options are: gaussian, gaussfunc, moffat, moffatfunc) ### NOTE: options with suffix 'func' fits a function to the pixels whereas those without this suffix takes into account the integrated function within each pixel (and is closer to truth)
settings['science']['extraction']['maxnumber'] = 999        # Maximum number of objects to extract in a science frame
settings['science']['extraction']['manual01'] = {}
settings['science']['extraction']['manual01']['frame'] = None
settings['science']['extraction']['manual01']['params'] = None # Info for desired extraction [det,x_pixel_location, y_pixel_location,[x_range,y_range]]


## Load up

In [60]:
# Bias
settings['bias'] = {}
settings['bias']['useframe'] = 'bias'
#
biasFrame = biasframe.BiasFrame(setup=setup, settings=settings)
msbias = biasFrame.master()
msbias.shape

[WARNING] :: processimages.py 108 __init__() - No information on the spectrograph was given.  Do not attempt to (re)process the images
[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2112, 350)

In [30]:
# Traceslits
settings.update(traceslits.default_settings().copy())
traceSlits = traceslits.TraceSlits(None, None, setup=setup, settings=settings)
traceSlits.master()

[INFO]    :: traceslits.py 520 _make_pixel_arrays() - Converting physical trace locations to nearest pixel
[INFO]    :: traceslits.py 527 _make_pixel_arrays() - Identifying the pixels belonging to each slit


True

In [13]:
# Wavelengths
waveImage = waveimage.WaveImage(setup=setup, settings=settings)
wave = waveImage.master()
wave.shape

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2048, 350)

In [17]:
# Tilts
waveTilts = wavetilts.WaveTilts(None, setup=setup, settings=settings)
tilts = waveTilts.master()
tilts.shape

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


(2048, 350)

In [62]:
# Flat
flatField = flatfield.FlatField(settings=settings, setup=setup)
msflat = flatField.master()

[WARNING] :: processimages.py 108 __init__() - No information on the spectrograph was given.  Do not attempt to (re)process the images
[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


In [33]:
# Maskslits
maskslits = np.array([False])

In [49]:
# datasec image
reload(arprocimg)
datasec_img = damg.get_datasec_trimmed(
    spectrograph, None, settings['detector']['num'], settings['detector'],
    naxis0=settings['detector']['naxis0'],
    naxis1=settings['detector']['naxis1'])
datasec_img.shape

(2048, 350)

In [89]:
# BPM
bpm = np.zeros_like(datasec_img)

## Instantiate

In [34]:
files = [pypdev_path+'/RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz']

In [300]:
reload(processimages)
reload(scienceimage)

sciI = scienceimage.ScienceImage(file_list=files,
                                    spectrograph=spectrograph,
                                     settings=settings,
                                     tilts=tilts,
                                     det=det,
                                     tslits_dict=traceSlits.tslits_dict,
                                     pixlocn=traceSlits.pixlocn,
                                     datasec_img=datasec_img,
                                     maskslits=maskslits,
                                     setup=setup,
                                         bpm=bpm
                                    )

## Process

In [301]:
_ = sciI._process(msbias, msflat, apply_gain=True)

[INFO]    :: arload.py 327 load_raw_frame() - Loading raw_file: /home/xavier/local/Python/PYPIT-development-suite//RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz
[INFO]    :: processimages.py 269 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: arprocimg.py 77 bias_subtract() - Subtracting bias image from raw frame
[WARNING] :: arprocimg.py 501 gain_frame() - Should probably be measuring the gain across the amplifier boundary


### View

In [302]:
sciI.show('sci')

## Variance image (Raw)

In [303]:
exptime = 1200. # s
dnoise = (settings['detector']['darkcurr'] * exptime/3600.0)
_ = sciI.build_rawvarframe(dnoise=dnoise)

[INFO]    :: processimages.py 390 build_rawvarframe() - Generate raw variance frame (from detected counts [flat fielded])


In [304]:
sciI.show('rawvar')

## CRs

In [305]:
_ = sciI.build_crmask()

[INFO]    :: arprocimg.py 232 lacosmic() - Detecting cosmic rays with the L.A.Cosmic algorithm
[WORK IN ]::
[PROGRESS]:: arprocimg.py 233 lacosmic() - Include these parameters in the settings files to be adjusted by the user
[INFO]    :: arprocimg.py 255 lacosmic() - Convolving image with Laplacian kernel
[INFO]    :: arprocimg.py 263 lacosmic() - Creating noise model
[INFO]    :: arprocimg.py 270 lacosmic() - Calculating Laplacian signal to noise ratio
[INFO]    :: arprocimg.py 278 lacosmic() - Selecting candidate cosmic rays
[INFO]    :: arprocimg.py 283 lacosmic() -  4470 candidate pixels
[INFO]    :: arprocimg.py 293 lacosmic() - Building fine structure image
[INFO]    :: arprocimg.py 302 lacosmic() - Removing suspected compact bright objects
[INFO]    :: arprocimg.py 312 lacosmic() -  3210 remaining candidate pixels
[INFO]    :: arprocimg.py 316 lacosmic() - Finding neighboring pixels affected by cosmic rays
[INFO]    :: arprocimg.py 337 lacosmic() -  5593 pixels detected as cosmi

In [306]:
sciI.show('crmasked')

## SkySub

In [307]:
settings['skysub'] = {}
settings['skysub']['perform']=True
settings['skysub']['method']= 'bspline'
settings['skysub']['bspline'] = {}
settings['skysub']['bspline']['everyn']=20
#
global_sky = sciI.global_skysub(settings)

[INFO]    :: scienceimage.py 189 _grab_varframe() - Using raw variance frame for sky sub
[INFO]    :: scienceimage.py 151 global_skysub() - Working on slit: 0
[INFO]    :: arskysub.py 38 bg_subtraction_slit() - Identifying pixels within each order
[INFO]    :: arskysub.py 42 bg_subtraction_slit() - Applying bad pixel mask
[INFO]    :: arskysub.py 60 bg_subtraction_slit() - Identifying pixels containing the science target
[WORK IN ]::
[PROGRESS]:: arskysub.py 61 bg_subtraction_slit() - Speed up this step with multi-processing
[INFO]    :: arskysub.py 130 bg_subtraction_slit() - Fitting sky background spectrum
[INFO]    :: arskysub.py 132 bg_subtraction_slit() - Using bspline sky subtraction


In [308]:
sciI.show('global')

In [309]:
sciI.show('image', image=sciI.sciframe-sciI.global_sky)

## Model Variance Frame (from the global sky)

In [310]:
_ = sciI.build_modelvar()

## Find Object

In [311]:
tracelist = sciI.find_objects()

[INFO]    :: scienceimage.py 192 _grab_varframe() - Using model variance frame for sky sub
[INFO]    :: artrace.py 278 trace_objects_in_slit() - Rectifying science frame
[INFO]    :: artrace.py 299 trace_objects_in_slit() - Estimating object profiles
[INFO]    :: artrace.py 368 trace_objects_in_slit() - Identifying objects that are significantly detected
[INFO]    :: artrace.py 451 trace_objects_in_slit() - Found 1 object
[INFO]    :: artrace.py 452 trace_objects_in_slit() - Tracing 1 object
[INFO]    :: artrace.py 491 trace_objects_in_slit() - Performing global trace to all objects
[INFO]    :: artrace.py 494 trace_objects_in_slit() - Constructing a trace for all objects
[INFO]    :: artrace.py 500 trace_objects_in_slit() - Converting object trace to detector pixels
[INFO]    :: artrace.py 90 trace_objbg_image() - Creating an image weighted by object pixels for 1 objects
[INFO]    :: artrace.py 99 trace_objbg_image() - Creating an image weighted by background pixels


In [312]:
tracelist[0].keys()

dict_keys(['nobj', 'traces', 'object', 'params', 'background'])

## Another round of skysub

In [313]:
_ = sciI.global_skysub(settings, use_tracemask=True)
_ = sciI.build_modelvar()

[INFO]    :: scienceimage.py 192 _grab_varframe() - Using model variance frame for sky sub
[INFO]    :: scienceimage.py 151 global_skysub() - Working on slit: 0
[INFO]    :: arskysub.py 38 bg_subtraction_slit() - Identifying pixels within each order
[INFO]    :: arskysub.py 42 bg_subtraction_slit() - Applying bad pixel mask
[INFO]    :: arskysub.py 60 bg_subtraction_slit() - Identifying pixels containing the science target
[WORK IN ]::
[PROGRESS]:: arskysub.py 61 bg_subtraction_slit() - Speed up this step with multi-processing
[INFO]    :: arskysub.py 130 bg_subtraction_slit() - Fitting sky background spectrum
[INFO]    :: arskysub.py 132 bg_subtraction_slit() - Using bspline sky subtraction


In [314]:
sciI.show('global')

In [315]:
sciI.show('skysub')